#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 3, 5)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,32,S,HREIT,"40,100",9.00,2,DOS,SET


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-2,ROUND,SET,"1,500",42.00,42.50,-0.25,-0.58%,2,Buy,"63,000.00"
7,B,TPIPL,-4,DOS,SET,"60,000",1.70,1.74,0.00,0.00%,2,Buy,"102,000.00"
10,B,WHART,-3,DOS,SET,"10,000",12.10,12.40,0.30,+2.48%,2,Buy,"121,000.00"
12,S,CPNCG,1,DOS,SET,"20,000",12.50,12.40,0.00,0.00%,2,Sell,"250,000.00"
14,S,HREIT,4,DOS,SET,"40,100",9.00,8.80,-0.25,-2.76%,2,Sell,"360,900.00"
16,S,JASIF,2,DOS,SET,"10,000",11.20,11.00,0.00,0.00%,2,Sell,"112,000.00"
19,S,PTT,8,5pct,SET50,"3,000",41.25,39.25,-0.75,-1.88%,2,Sell,"123,750.00"
21,S,SINGER,2,15pct,SET100,"1,500",49.00,48.50,0.75,+1.57%,2,Sell,"73,500.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,MAKRO,-2,ROUND,SET,"1,500",42.00,42.50,-0.25,-0.58%,2,Buy,"63,000.00"
B,ASP,-3,3L,SET,"30,000",3.80,3.86,-0.06,-1.53%,1,Buy,"114,000.00"
B,MEGA,-3,ROUND,SET100,"1,500",43.00,43.75,-0.75,-1.69%,1,Buy,"64,500.00"
B,GLOBAL,-10,nan,SET50,"9,000",19.40,20.40,0.10,+0.49%,1,Buy,"174,600.00"
B,BAM,-13,nan,SET100,"10,000",20.50,21.80,-0.20,-0.91%,0,Buy,"205,000.00"


### End of Tables in the process

### Print to verify before upload file

In [11]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [12]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [51]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,TPIPL,"60,000",1.70,"102,000.00",DOS,SET,2
2,B,WHART,"10,000",12.10,"121,000.00",DOS,SET,2
3,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
4,S,HREIT,"40,100",9.00,"360,900.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2
7,S,SINGER,"1,500",49.00,"73,500.00",15pct,SET100,2


In [52]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    286000.0
S    920150.0
Name: amount, dtype: float64

In [53]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,TPIPL,"60,000",1.70,"102,000.00",DOS,SET,2
2,B,WHART,"10,000",12.10,"121,000.00",DOS,SET,2


In [54]:
cash = 1_092_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(1092000, 286000.0, 806000.0)

In [55]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
3,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
4,S,HREIT,"40,100",9.00,"360,900.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2
7,S,SINGER,"1,500",49.00,"73,500.00",15pct,SET100,2


In [56]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(1092000, 920150.0, 2012150.0)

### After call ord-log (must call everytime that orders change)

In [57]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [58]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [59]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,ASP,-3,3L,SET,"30,000",3.80,3.86,-0.06,-1.53%,1,Buy,"114,000.00"
3,B,MAKRO,-2,ROUND,SET,"1,500",42.00,42.50,-0.25,-0.58%,2,Buy,"63,000.00"
4,B,MEGA,-3,ROUND,SET100,"1,500",43.00,43.75,-0.75,-1.69%,1,Buy,"64,500.00"
6,B,TMT,-5,ROUND,SET,"18,000",11.00,11.50,0.20,+1.77%,1,Buy,"198,000.00"
7,B,TPIPL,-4,DOS,SET,"60,000",1.70,1.74,0.00,0.00%,2,Buy,"102,000.00"
9,B,TU,-5,3L,SET50,"9,000",19.30,19.80,-0.10,-0.50%,1,Buy,"173,700.00"
10,B,WHART,-3,DOS,SET,"10,000",12.10,12.40,0.30,+2.48%,2,Buy,"121,000.00"


In [60]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [61]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
12,S,CPNCG,1,DOS,SET,"20,000",12.50,12.40,0.00,0.00%,2,Sell,"250,000.00"
14,S,HREIT,4,DOS,SET,"40,100",9.00,8.80,-0.25,-2.76%,2,Sell,"360,900.00"
16,S,JASIF,2,DOS,SET,"10,000",11.20,11.00,0.00,0.00%,2,Sell,"112,000.00"
21,S,SINGER,2,15pct,SET100,"1,500",49.00,48.50,0.75,+1.57%,2,Sell,"73,500.00"


In [62]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
0,B,ASP,-3,3L,SET,30000,3.8,3.86,-0.06,-1.53%,1,Buy,114000.0
3,B,MAKRO,-2,ROUND,SET,1500,42.0,42.50,-0.25,-0.58%,2,Buy,63000.0
4,B,MEGA,-3,ROUND,SET100,1500,43.0,43.75,-0.75,-1.69%,1,Buy,64500.0
6,B,TMT,-5,ROUND,SET,18000,11.0,11.50,0.20,+1.77%,1,Buy,198000.0
7,B,TPIPL,-4,DOS,SET,60000,1.7,1.74,0.00,0.00%,2,Buy,102000.0
9,B,TU,-5,3L,SET50,9000,19.3,19.80,-0.10,-0.50%,1,Buy,173700.0
10,B,WHART,-3,DOS,SET,10000,12.1,12.40,0.30,+2.48%,2,Buy,121000.0
12,S,CPNCG,1,DOS,SET,20000,12.5,12.40,0.00,0.00%,2,Sell,250000.0
14,S,HREIT,4,DOS,SET,40100,9.0,8.80,-0.25,-2.76%,2,Sell,360900.0
16,S,JASIF,2,DOS,SET,10000,11.2,11.00,0.00,0.00%,2,Sell,112000.0


In [63]:
df[mb | ms].shape[0]

11

In [65]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
1,BAM,Buy,"205,000.00",-13
6,TMT,Buy,"198,000.00",-5
8,TQM,Buy,"193,500.00",-8
2,GLOBAL,Buy,"174,600.00",-10
9,TU,Buy,"173,700.00",-5


### Select source of orders between these two

In [66]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
0,Buy,ASP,"30,000",3.80,1,-3,3.86,-0.06,-1.53%,3L,SET
3,Buy,MAKRO,"1,500",42.00,2,-2,42.50,-0.25,-0.58%,ROUND,SET
4,Buy,MEGA,"1,500",43.00,1,-3,43.75,-0.75,-1.69%,ROUND,SET100
6,Buy,TMT,"18,000",11.00,1,-5,11.50,0.20,+1.77%,ROUND,SET
7,Buy,TPIPL,"60,000",1.70,2,-4,1.74,0.00,0.00%,DOS,SET
9,Buy,TU,"9,000",19.30,1,-5,19.80,-0.10,-0.50%,3L,SET50
10,Buy,WHART,"10,000",12.10,2,-3,12.40,0.30,+2.48%,DOS,SET
12,Sell,CPNCG,"20,000",12.50,2,1,12.40,0.00,0.00%,DOS,SET
14,Sell,HREIT,"40,100",9.00,2,4,8.80,-0.25,-2.76%,DOS,SET
16,Sell,JASIF,"10,000",11.20,2,2,11.00,0.00,0.00%,DOS,SET


In [39]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,MAKRO,"1,500",42.00,2,-3,42.75,-0.25,-0.58%,ROUND,SET
6,Buy,STA,"2,500",25.75,2,-1,26.00,-0.50,-1.89%,RD25pct,SET50
8,Buy,TPIPL,"60,000",1.70,2,-4,1.74,-0.01,-0.57%,nan,SET
11,Buy,WHART,"10,000",12.00,2,-1,12.10,-0.30,-2.42%,DOS,SET
13,Sell,CPNCG,"20,000",12.50,2,1,12.40,0.10,+0.81%,DOS,SET
15,Sell,HREIT,"40,100",9.00,2,-1,9.05,0.00,0.00%,DOS,SET
17,Sell,JASIF,"10,000",11.20,2,2,11.00,-0.20,-1.79%,DOS,SET
20,Sell,PTT,"3,000",41.25,2,5,40.00,0.25,+0.63%,5pct,SET50


In [67]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [85]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,MEGA,"1,500",43.00,"64,500.00",6L,SET100,2
2,B,STA,"2,500",25.75,"64,375.00",RD25pct,SET50,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,MEGA,"1,500",47.25,"70,875.00",10pct,SET100,2
7,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2


### Evening Process

In [86]:
new_active = 1
old_active = 2

In [87]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

8

In [88]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'MAKRO', 'STA', 'TPIPL', 'CPNCG', 'JASIF', 'PTT'"

In [89]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

6

In [66]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process